STEPS INVOLVED


In [ ]:
#Setup and Import Libraries
#Load and Visualize Data
#Preprocess Images
#Build and Tune the CNN Model using KerasTuner
#Train and Evaluate the Model
#Create a Streamlit App

In [ ]:
# Install necessary libraries
!pip install streamlit
!pip install keras-tuner
!pip install opencv-python
!pip install seaborn
!pip install gdown

# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import keras_tuner as kt
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from zipfile import ZipFile

# Streamlit imports
import streamlit as st


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
# Specify the path to your zip file
zip_file_path = '/content/archive.zip'

# Extract the zip file
!unzip -q {zip_file_path} -d gtsrb_data

# Load the data
images = []
labels = []
def load_data(data_dir):
    # List the contents of the data_dir
    try:
        dir_contents = os.listdir(data_dir)
    except FileNotFoundError:
        print(f"Error: Directory {data_dir} not found.")
        return np.array(images), np.array(labels)

    # Iterate through each class directory
    for class_dir in dir_contents:
        class_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                img_path = os.path.join(class_path, img_file)
                img = cv2.imread(img_path)

                img = img/255
                if img is not None:  # Check if the image is read correctly
                    img = cv2.resize(img, (32, 32))
                    images.append(img)
                    labels.append(int(class_dir))
                else:
                    print(f"Failed to read {img_path}")
        else:
            print(f"Skipping non-directory {class_path}")
    return np.array(images), np.array(labels)

# Check the contents of the unzipped directory to confirm the structure
!ls gtsrb_data

# Assuming 'Train' directory exists after unzipping
data_dir = 'gtsrb_data/Train'
if not os.path.exists(data_dir):
    print(f"Directory {data_dir} does not exist. Please check the extracted contents.")
else:
    images, labels = load_data(data_dir)

    print(images)

    # Check if data is loaded correctly
    if len(images) == 0 or len(labels) == 0:
        print("No images or labels loaded. Please check the dataset structure.")
    else:
        print(f"Loaded {len(images)} images with {len(labels)} labels.")

        # Visualize the data
        fig, axes = plt.subplots(3, 3, figsize=(10, 10))
        axes = axes.ravel()

        for i in np.arange(0, 9):
            axes[i].imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for correct display
            axes[i].set_title(f"Class: {labels[i]}")
            axes[i].axis('off')

        plt.subplots_adjust(hspace=0.5)
        plt.show()





In [ ]:
print(images)

In [ ]:


# One-hot encode the labels
num_classes = len(np.unique(labels))
labels = to_categorical(labels, num_classes)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")



In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                     kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                     activation='relu',
                     input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
                     kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(hp.Int('dense_units', min_value=32, max_value=128, step=16), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='traffic_signs')

tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))
best_model = tuner.get_best_models(num_models=1)[0]



NameError: name 'num_classes' is not defined

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the best model
model = build_model(best_hps)

# Train the model
history = model.fit(images, labels_categorical, epochs=20, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(images, labels_categorical)
print(f"Test accuracy: {accuracy}")


In [ ]:
model.save('traffic_sign_classifier.h5')


In [ ]:

from PIL import Image

# Load the trained model
model = tf.keras.models.load_model('traffic_sign_classifier.h5')

# Class labels
class_names = ["Speed limit (20km/h)", "Speed limit (30km/h)", "Speed limit (50km/h)",
               "Speed limit (60km/h)", "Speed limit (70km/h)", "Speed limit (80km/h)",
               "End of speed limit (80km/h)", "Speed limit (100km/h)", "Speed limit (120km/h)",
               "No passing", "No passing for vehicles over 3.5 metric tons", "Right-of-way at the next intersection",
               "Priority road", "Yield", "Stop", "No vehicles", "Vehicles over 3.5 metric tons prohibited",
               "No entry", "General caution", "Dangerous curve to the left", "Dangerous curve to the right",
               "Double curve", "Bumpy road", "Slippery road", "Road narrows on the right", "Road work",
               "Traffic signals", "Pedestrians", "Children crossing", "Bicycles crossing", "Beware of ice/snow",
               "Wild animals crossing", "End of all speed and passing limits", "Turn right ahead", "Turn left ahead",
               "Ahead only", "Go straight or right", "Go straight or left", "Keep right", "Keep left",
               "Roundabout mandatory", "End of no passing", "End of no passing by vehicles over 3.5 metric tons"]

# Function to preprocess image
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (32, 32))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

st.title("Traffic Sign Classifier")
st.write("Upload an image of a traffic sign to classify it.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = np.array(Image.open(uploaded_file))
    st.image(image, caption='Uploaded Image', use_column_width=True)

    # Preprocess the image
    preprocessed_image = preprocess_image(image)

    # Predict the class
    prediction = model.predict(preprocessed_image)
    class_index = np.argmax(prediction)
    class_label = class_names[class_index]

    st.write(f"Predicted Class: {class_label}")


In [ ]:
streamlit run app.py


SyntaxError: invalid syntax (<ipython-input-7-718866ff34b9>, line 1)